In [ ]:
#import required packages

import math
import torch
import torch.nn as nn
import cv2
import numpy as np
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import StepLR
import json
from time import time
import argparse
import sys
sys.argv=['']
import os
import h5py
from torch.utils.data import DataLoader, TensorDataset,random_split
import glob
import natsort
from PIL import Image
#import matplotlib.pyplot as plt
import time
import numpy as np
import torchvision.transforms as T
from torchvision import datasets
from torchvision.utils import make_grid
from torchvision.utils import save_image
import random
import PIL.ImageOps
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import matplotlib.pyplot as plt
from torch.nn.modules.utils import _quadruple
from sklearn.model_selection import KFold
from scipy.ndimage import gaussian_filter

In [ ]:
# Set the seed for reproducibility
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
g_cpu = torch.Generator().manual_seed(seed)
g_cuda = torch.Generator(device='cuda').manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Ensure deterministic behavior for CUDA operations
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

## Loading and preprocessing Dataset

In [ ]:
# import required libraries


class LoadDataset:
    def __init__(self, dataset_path,transform_layout,in_channels,out_channels,n_images,inputdata_type,outputdata_type):
        '''
        This class reads the dataset from specified folder.
        '''
        self.dataset_path=dataset_path
        self.transform_layout=transform_layout
        self.in_channels=in_channels
        self.out_channels=out_channels
        self.n_images=n_images
        self.inputdata_type=inputdata_type
        self.outputdata_type=outputdata_type



        self.U_input, self.V_input = [], []
        self.U_output, self.V_output = [], []

        self.read_dataset()
    def read_dataset(self):
        '''
        This function reads the dataset from the specified folder.
        '''
        # Read the dataset from the specified folder
        if self.in_channels==2:
            if self.inputdata_type=='Larsen':
                self.U_input=natsort.natsorted(glob.glob(os.path.join(self.dataset_path, '*u_flow.npy')))
                self.V_input=natsort.natsorted(glob.glob(os.path.join(self.dataset_path, '*v_flow.npy')))
            else:
                print('The dataset type is not supported')
        else:
            print('The number of input channels is not supported')

        if self.out_channels==2:
            if  self.outputdata_type=='CFD':
                self.U_output= natsort.natsorted(glob.glob(os.path.join(self.dataset_path, '*_sw.npy')))
                self.V_output= natsort.natsorted(glob.glob(os.path.join(self.dataset_path, '*_cw.npy')))
                self.P= natsort.natsorted(glob.glob(os.path.join(self.dataset_path, '*_p.npy')))
                self.K= natsort.natsorted(glob.glob(os.path.join(self.dataset_path, '*_k.npy')))
                self.Omega= natsort.natsorted(glob.glob(os.path.join(self.dataset_path, '*_omega.npy')))
                self.Coordin= natsort.natsorted(glob.glob(os.path.join(self.dataset_path, '*_layout_pixels.npy')))
            else:
                print('The dataset type is not supported')

        return



    def __getitem__(self, index):
        '''
        This function returns the data at the specified index.
        '''
        if self.n_images:
            if index >= self.n_images:
                raise Exception('CustomDataSet:Error: Index out of bounds inside __get_item__')

        if self.in_channels==2:
            input_U = np.load(self.U_input[index])
#             input_U=self.gaussian_smooth(input_U)
            input_U=torch.from_numpy(input_U.copy()).unsqueeze(0)

            input_V = np.load(self.V_input[index])
#             input_V=self.gaussian_smooth(input_V)
            input_V=torch.from_numpy(input_V.copy()).unsqueeze(0)

            input_data=torch.cat((input_U,input_V))
        if self.out_channels==2:
            output_U = np.load(self.U_output[index])
            output_U=np.rot90(output_U, axes=(1, 0))

            ############################################################################################################
            P = np.load(self.P[index])
            P=np.rot90(P, axes=(1, 0))
            P=torch.from_numpy(P.copy()).unsqueeze(0)
            P=F.interpolate(P.unsqueeze(0) , size=(400), mode='bilinear', align_corners=True).squeeze(0)
            ############################################################################################################
            K = np.load(self.K[index])
            K=np.rot90(K, axes=(1, 0))
            K=torch.from_numpy(K.copy()).unsqueeze(0)
            K=F.interpolate(K.unsqueeze(0) , size=(400), mode='bilinear', align_corners=True).squeeze(0)
            ############################################################################################################
            Omega = np.load(self.Omega[index])
            Omega=np.rot90(Omega, axes=(1, 0))
            Omega=torch.from_numpy(Omega.copy()).unsqueeze(0)
            Omega=F.interpolate(Omega.unsqueeze(0) , size=(400), mode='bilinear', align_corners=True).squeeze(0)
            ############################################################################################################
            output_V = np.load(self.V_output[index])
            output_V=np.rot90(output_V, axes=(1, 0))

            #################################################################################
            force=np.zeros((400,400))
            loc_layout=self.Coordin[index]
            lay=np.load(loc_layout)
            cp=0.5926
            ct=0.889
            a=1-(cp/ct)
            for i in range(lay.shape[0]):


                U=np.sqrt(output_U[int(lay[i][1]),int(lay[i][0])-5]**2+output_V[int(lay[i][1]),int(lay[i][0])-5]**2)

                Force=(2*U**2*a*(1-a))/4

                force[int(lay[i][1])-4:int(lay[i][1])+4,int(lay[i][0])]=Force
            force_tensor=torch.from_numpy(force.copy())
            force_tensor=force_tensor.unsqueeze(0)

            output_V=torch.from_numpy(output_V.copy()).unsqueeze(0)
            output_V=F.interpolate(output_V.unsqueeze(0), size=(400), mode='bilinear', align_corners=True).squeeze(0)
            output_U=torch.from_numpy(output_U.copy()).unsqueeze(0)
            output_U=F.interpolate(output_U.unsqueeze(0) , size=(400), mode='bilinear', align_corners=True).squeeze(0)

            output_data=torch.cat((output_U,output_V))


        return input_data,output_data,P,K,Omega,force_tensor



    def __len__(self):
        '''
        This function returns the length of the dataset.
        '''
        return self.n_images


class Preprocessor:
    def __init__(self, method='minmax'):
        self.method = method
        self.global_min_U_input = None
        self.global_max_U_input = None
        self.global_min_V_input = None
        self.global_max_V_input = None
        self.global_mean_U_input = None
        self.global_std_U_input = None
        self.global_mean_V_input = None
        self.global_std_V_input = None
        self.global_min_U_output = None
        self.global_max_U_output = None
        self.global_min_V_output = None
        self.global_max_V_output = None
        self.global_mean_U_output = None
        self.global_std_U_output = None
        self.global_mean_V_output = None
        self.global_std_V_output = None

    def fit(self, dataset):
        '''
        This function calculates the global min and max values (or mean and std) for input_U and input_V.
        '''
        all_U_input = []
        all_V_input = []
        all_U_output = []
        all_V_output = []

        for i in range(len(dataset)):
            input_data, output_data,P,K,Omega,force_tensor = dataset[i]
            input_U = input_data[0].numpy()
            input_V = input_data[1].numpy()
            output_U = output_data[0].numpy()
            output_V = output_data[1].numpy()
            all_U_input.append(input_U)
            all_V_input.append(input_V)
            all_U_output.append(output_U)
            all_V_output.append(output_V)

        all_U_input = np.concatenate(all_U_input)
        all_V_input = np.concatenate(all_V_input)
        all_U_output = np.concatenate(all_U_output)
        all_V_output = np.concatenate(all_V_output)

        if self.method == 'minmax':
            self.global_min_U_input = all_U_input.min()
            self.global_max_U_input = all_U_input.max()
            self.global_min_V_input = all_V_input.min()
            self.global_max_V_input = all_V_input.max()
            self.global_min_U_output = all_U_output.min()
            self.global_max_U_output = all_U_output.max()
            self.global_min_V_output = all_V_output.min()
            self.global_max_V_output = all_V_output.max()

        elif self.method == 'standardization':
            self.global_mean_U_input = all_U_input.mean()
            self.global_std_U_input = all_U_input.std()
            self.global_mean_V_input = all_V_input.mean()
            self.global_std_V_input = all_V_input.std()
            self.global_mean_U_output = all_U_output.mean()
            self.global_std_U_output = all_U_output.std()
            self.global_mean_V_output = all_V_output.mean()
            self.global_std_V_output = all_V_output.std()




    def transform(self, input_data, output_data):
        '''
        This function applies the specified normalization method on the data.
        '''
        if self.method == 'minmax' and (self.global_min_U_input is None or self.global_max_U_input is None):
            raise ValueError("Min and max values for minmax scaling are not set. Please call the fit method first.")
        elif self.method == 'standardization' and (self.global_mean_U_input is None or self.global_std_U_input is None):
            raise ValueError("Mean and std values for standardization are not set. Please call the fit method first.")

        input_U, input_V = input_data[0].numpy(), input_data[1].numpy()
        output_U, output_V = output_data[0].numpy(), output_data[1].numpy()

        if self.method == 'standardization':
            input_U = (input_U - self.global_mean_U_input) / self.global_std_U_input
            input_V = (input_V - self.global_mean_V_input) / self.global_std_V_input
            output_U = (output_U - self.global_mean_U_output) / self.global_std_U_output
            output_V = (output_V - self.global_mean_V_output) / self.global_std_V_output
        elif self.method == 'minmax':

            input_U = (input_U - self.global_min_U_input) / (self.global_max_U_input - self.global_min_U_input)

            input_V = -1 + 2 * (input_V - self.global_min_V_input) / (self.global_max_V_input - self.global_min_V_input)




        input_data = torch.from_numpy(np.stack((input_U, input_V)))
        output_data = torch.from_numpy(np.stack((output_U, output_V)))

        return input_data, output_data

    def fit_transform(self, dataset):
        '''
        This function first fits the preprocessor on the dataset and then transforms it.
        '''
        self.fit(dataset)
        transformed_dataset = []

        for i in range(len(dataset)):
            input_data, output_data,P,K,Omega,force_tensor = dataset[i]
            input_data, output_data = self.transform(input_data, output_data)
            transformed_dataset.append((input_data, output_data,P,K,Omega,force_tensor ))

        return transformed_dataset

    def transform_only(self, dataset):
        '''
        This function transforms the dataset without fitting, using already fitted values.
        '''
        transformed_dataset = []

        for i in range(len(dataset)):
            input_data, output_data,P,K,Omega,force_tensor = dataset[i]
            input_data, output_data = self.transform(input_data, output_data)
            transformed_dataset.append((input_data, output_data,P,K,Omega,force_tensor))

        return transformed_dataset

    def get_max_min(self,dataset):
        return self.global_max_U_output,self.global_min_U_output,self.global_max_V_output,self.global_min_V_output



import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np
transforms_list = [
                   T.Grayscale(),

                   T.ToTensor()]
transform_layout = T.Compose(transforms_list)
dataset_path=r'.\data\New_train_1'

in_channels=2
out_channels=2
n_images=190
inputdata_type='Larsen'
outputdata_type='CFD'
dataset=LoadDataset(dataset_path,transform_layout,in_channels,out_channels,n_images,inputdata_type,outputdata_type)



In [ ]:
preprocessor = Preprocessor(method='minmax')
preprocessor.fit(dataset)
U_max,U_min,V_max,V_min=preprocessor.get_max_min(dataset)
transformed_dataset = preprocessor.fit_transform(dataset)
train_data=transformed_dataset


In [ ]:
dataset_path=r'.\data\valid1'
in_channels=2
out_channels=2
n_images=8
inputdata_type='Larsen'
outputdata_type='CFD'
dataset_valid=LoadDataset(dataset_path,transform_layout,in_channels,out_channels,n_images,inputdata_type,outputdata_type)



In [ ]:

transformed_dataset_valid = preprocessor.transform_only(dataset_valid)
valid_data=transformed_dataset_valid

## Dataset Visualizzation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(10, 10))
for i in range(3):
    # Plot input
    ax = fig.add_subplot(3, 3, i + 1)
    img = ax.imshow(train_data.__getitem__(i)[1][0, :, :].squeeze(0), cmap='jet')
    ax.set_title('Inputs_{}'.format(i + 1))
    ax.axis("off")
    fig.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

    # Plot Target_U
    ax = fig.add_subplot(3, 3, i + 1 + 3)
    img = ax.imshow(train_data.__getitem__(i)[1][1, :, :], cmap='jet')
    ax.set_title('Target_U{}'.format(i + 1))
    ax.axis("off")
    fig.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

    # Plot Target_V
    ax = fig.add_subplot(3, 3, i + 1 + 6)
    img = ax.imshow(train_data.__getitem__(i)[2][0, :, :], cmap='jet')
    ax.set_title('Target_V{}'.format(i + 1))
    ax.axis("off")
    fig.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()


In [ ]:

nvalid=len(valid_data)

ntrain=len(train_data)
print('Number of train data:{}'.format(ntrain ))
print('Number of valid data:{}'.format(nvalid ))

## Evaluation Function

In [ ]:
# Evaluating the validation and test set
def evaluate(model,epoch,valid_set):



    val_loss=0


    device='cuda'




    for step,(inputs,target,P,K,Omega,force) in enumerate(valid_set):


        model.eval()

            #############################################
        inputs  = inputs.to(device, dtype=torch.float)

        targets= target.to(device, dtype=torch.float)



        outputs = model(inputs)
        loss_MSE=F.mse_loss(outputs, targets, reduction='sum')


        valid_loss_log=torch.log(1+loss_MSE)
#



        validation_loss=loss_MSE.item()
        validation_loss_log=valid_loss_log.item()






    return validation_loss,validation_loss_log

## Visualzation function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def visual( img_target,img_pred, gpu=0, flag_torch=0, sv=False):
    if gpu:

        img_target = img_target.cpu().detach()

        img_pred = img_pred.cpu().detach()



    if flag_torch:
#         mass_target = mass_target.numpy()
        img_target = img_target.numpy()

        img_pred = img_pred.numpy()


    if flag_torch == 2:
        img_target = np.transpose(img_target[:, :, :, :, :], [0, 4, 2, 3, 1]).squeeze()
        img_pred = np.transpose(img_pred[:, :, :, :, :], [0, 4, 2, 3, 1]).squeeze()


#     mass_out = np.transpose(mass_out[:5, :, :, :], [0, 2, 3, 1])
    img_target = np.transpose(img_target[:5, :, :, :], [0, 2, 3, 1])
    img_pred = np.transpose(img_pred[:5, :, :, :], [0, 2, 3, 1])


    fig = plt.figure(figsize=(10, 10))
    for i in range(3):


        ax = fig.add_subplot(6, 3, i + 1)
        im = ax.imshow(abs(img_target[:,:,:,0][i]-img_pred[:,:,:,0][i]), cmap='jet',vmin=0,vmax=0.1)
        ax.set_title('Error_U{}'.format(i+1))
        ax.axis("off")
        cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

        ax = fig.add_subplot(6, 3, i +3+ 1)
        im = ax.imshow(abs(img_target[:,:,:,1][i]-img_pred[:,:,:,1][i]), cmap='jet',vmin=0,vmax=0.1)
        ax.set_title('Error_V{}'.format(i+1))
        ax.axis("off")
        cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)



        ax = fig.add_subplot(6, 3, i + 1 + 6)
        im = (img_target[:,:,:,0][i])
        im = ax.imshow(im, cmap='jet')
        ax.set_title('Target_U{}'.format(i+1))
        ax.axis("off")
        cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

        ax = fig.add_subplot(6, 3, i + 1 + 9)
        im = (img_pred[:,:,:,0][i])
        im = ax.imshow(im, cmap='jet')
        ax.set_title('Prediction_U{}'.format(i+1))
        ax.axis("off")
        cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

        ax = fig.add_subplot(6, 3, i + 1 + 12)
        im = (img_target[:,:,:,1][i])
        im = ax.imshow(im, cmap='jet')
        ax.set_title('Target_V{}'.format(i+1))
        ax.axis("off")
        cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

        ax = fig.add_subplot(6, 3, i + 1 + 15)
        im = (img_pred[:,:,:,1][i])
        im = ax.imshow(im, cmap='jet')
        ax.set_title('Prediction_V{}'.format(i+1))
        ax.axis("off")
        cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()
#     if sv:
#         fig.savefig('temp.png')

# Example usage
# visual(img_layout, img_target, img_pred, gpu=1, flag_torch=1, sv=True)


## Network

In [ ]:

#Convolutional layers in the dense blocks

class conv_layer(nn.Module):
    def __init__(self,init_features,kernel_conv,stride_conv,padding_conv,growth_rate,bn_size,drop_out,bottleneck):
        super(). __init__()
        if bottleneck and init_features>growth_rate*bn_size:
            self.conv=nn.Sequential(
                nn.BatchNorm2d(init_features),
                nn.ReLU(inplace='True'),
                nn.Conv2d(init_features,growth_rate*bn_size,kernel_size=1,stride=1,padding=0,bias=False),
                nn.BatchNorm2d(growth_rate*bn_size),
                nn.ReLU(inplace='True'),
                nn.Conv2d(growth_rate*bn_size,growth_rate,kernel_conv,stride_conv,padding_conv,bias=False)
            )
        else:
            self.conv=nn.Sequential(
                nn.BatchNorm2d(init_features),
                nn.ReLU(inplace='True'),
                nn.Conv2d(init_features,growth_rate,kernel_conv,stride_conv,padding_conv,bias=False)
            )
        self.drop_out=drop_out

    def forward(self,x):
        out=self.conv(x)
        if self.drop_out>0:
            out = F.dropout(out, p=self.drop_out)
        return torch.cat([x,out],1)

# Dense blocks

class Dense_block(nn.Module):
    def __init__(self,init_features,kernel_conv,stride_conv,padding_conv,conv_layer,n_layers,growth_rate,bn_size,drop_out,bottleneck):
        super().__init__()
        self.layer = self._make_layer(init_features,kernel_conv,stride_conv,padding_conv,n_layers,conv_layer,growth_rate,bn_size,drop_out,bottleneck)
    def _make_layer(self,init_features,kernel_conv,stride_conv,padding_conv,n_layers,conv_layer,growth_rate,bn_size,drop_out,bottleneck):
        layers=[]
        for i in range(n_layers):
            layers.append(conv_layer(init_features+i*growth_rate,kernel_conv,stride_conv,padding_conv,growth_rate,bn_size,drop_out,bottleneck))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)
#Encoders and decoders

class Encode_Decode(nn.Module):
    def __init__(self,init_features,kernel,stride,padding,output_padding,out_features,growth_rate,drop_out,down,bottleneck):
        super().__init__()
        self.drop_out=drop_out
        if down:
            if bottleneck:
                self.block1=nn.Sequential(nn.BatchNorm2d(init_features),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(init_features,out_features,kernel_size=1,stride=1,padding=0,bias=False),
                    nn.BatchNorm2d(out_features),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(out_features,out_features,kernel,stride,padding,bias=False)
                    )
            else:
                self.block1=nn.Sequential(nn.BatchNorm2d(init_features),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(init_features,out_features,kernel,stride,padding,bias=False)
                    )
        else:
            if bottleneck:
                self.block1=nn.Sequential(nn.BatchNorm2d(init_features),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(init_features,out_features,kernel_size=1,stride=1,padding=0,bias=False),
                    nn.BatchNorm2d(out_features),
                    nn.ReLU(inplace=True),
                    nn.ConvTranspose2d(out_features,out_features,kernel,stride,padding,output_padding,bias=False)
                    )

            else:
                self.block1=nn.Sequential(nn.BatchNorm2d(init_features),
                    nn.ReLU(inplace=True),
                    nn.ConvTranspose2d(out_features,out_features,kernel,stride,padding,output_padding,bias=False)
                                         )
    def forward(self,x):
        out=self.block1(x)
        if self.drop_out>0:
            out = F.dropout(out, p=self.drop_out)
        return out

class last_decode(nn.Module):
    def __init__(self,init_features,out_channels,kernel_size,stride,padding,output_padding,drop_out):
        super().__init__()
        self.norm1=nn.BatchNorm2d(init_features)
        self.relu1=nn.ReLU(inplace=True)
        self.conv1=nn.Conv2d(init_features,init_features//2,kernel_size=1,stride=1,padding=0,bias=False)
        self.norm2=nn.BatchNorm2d(init_features//2)
        self.relu2=nn.ReLU(inplace=True)
        self.conv2=nn.ConvTranspose2d(init_features//2,out_channels,kernel_size,stride,padding,output_padding,bias=False)
        self.drop_out=drop_out
    def forward(self,x):
        out=self.conv1(self.relu1(self.norm1(x)))
        out=self.conv2(self.relu2(self.norm2(out)))
        if self.drop_out>0:
            out = F.dropout(out, p=self.drop_out)
        return out



In [ ]:

class Dense_C8_one_layer_Added1(nn.Module):
    def __init__(self,in_channels,kernel,stride,padding,output_padding,out_channels_V,out_channels_P,init_features,growth_rate,bn_size,bottleneck,drop_out):
        super().__init__()
        block=conv_layer
        self.conv1=nn.Conv2d(in_channels,init_features,kernel[3], stride[3], padding[3],bias=False)

        n_layers=2
        self.dense_block1=Dense_block(init_features,kernel[0],stride[0],padding[0],block,n_layers,growth_rate,bn_size,drop_out,bottleneck)
        in_features1=init_features+growth_rate*n_layers
        self.encode1=Encode_Decode(in_features1,kernel[1],stride[1],padding[1],output_padding[0],in_features1//2,growth_rate,drop_out,down=True,bottleneck=True)

        n_layers=2
        self.dense_block2=Dense_block(in_features1//2,kernel[0],stride[0],padding[0],block,n_layers,growth_rate,bn_size,drop_out,bottleneck)
        in_features2=in_features1//2+growth_rate*n_layers
        self.encode2=Encode_Decode(in_features2,kernel[1],stride[1],padding[1],output_padding[0],in_features2//2,growth_rate,drop_out,down=True,bottleneck=True)

        n_layers=2
        self.dense_block3=Dense_block(in_features2//2,kernel[0],stride[0],padding[0],block,n_layers,growth_rate,bn_size,drop_out,bottleneck)
        in_features3=in_features2//2+growth_rate*n_layers
        self.encode3=Encode_Decode(in_features3,kernel[1],stride[1],padding[1],output_padding[0],in_features3//2,growth_rate,drop_out,down=True,bottleneck=True)
        #####
        n_layers=2
        self.dense_block4=Dense_block(in_features3//2,kernel[0],stride[0],padding[0],block,n_layers,growth_rate,bn_size,drop_out,bottleneck)
        in_features4=in_features3//2+growth_rate*n_layers
        self.encode4=Encode_Decode(in_features4,kernel[1],stride[1],padding[1],output_padding[0],in_features4//2,growth_rate,drop_out,down=True,bottleneck=True)
        #####
        n_layers=4
        self.dense_block5=Dense_block(in_features4//2,kernel[0],stride[0],padding[0],block,n_layers,growth_rate,bn_size,drop_out,bottleneck)
        in_features11=in_features4//2+growth_rate*n_layers
        self.decode1=Encode_Decode(in_features11,kernel[1],stride[1],padding[1],output_padding[1],in_features11//2,growth_rate,drop_out,down=False,bottleneck=True)

        n_layers=2
        in_features=in_features11//2+in_features3//2
        self.dense_block11=Dense_block(in_features,kernel[0],stride[0],padding[0],block,n_layers,growth_rate,bn_size,drop_out,bottleneck)
        in_features22=in_features+growth_rate*n_layers
        self.decode2=Encode_Decode(in_features22,kernel[1],stride[1],padding[1],output_padding[2],in_features22//2,growth_rate,drop_out,down=False,bottleneck=True)

        n_layers=2
        in_features=in_features22//2+in_features2//2
        self.dense_block22=Dense_block(in_features,kernel[0],stride[0],padding[0],block,n_layers,growth_rate,bn_size,drop_out,bottleneck)
        in_features33=in_features+growth_rate*n_layers
        self.decode3=Encode_Decode(in_features33,kernel[1],stride[1],padding[1],output_padding[2],in_features33//2,growth_rate,drop_out,down=False,bottleneck=True)

        #####
        n_layers=2
        in_features=in_features33//2+in_features1//2
        self.dense_block33=Dense_block(in_features,kernel[0],stride[0],padding[0],block,n_layers,growth_rate,bn_size,drop_out,bottleneck)
        in_features44=in_features+growth_rate*n_layers
        self.decode4=Encode_Decode(in_features44,kernel[1],stride[1],padding[1],output_padding[2],in_features44//2,growth_rate,drop_out,down=False,bottleneck=True)
        ####

        n_layers=2

        in_features=in_features44//2
        self.dense_block44=Dense_block(in_features,kernel[0],stride[0],padding[0],block,n_layers,growth_rate,bn_size,drop_out,bottleneck)
        in_features_last=in_features+growth_rate*n_layers
        self.last_V=last_decode(in_features_last,out_channels_V,kernel[2],stride[2],padding[2],output_padding[3],drop_out=0)






    def forward(self,x):
        out1=self.conv1(x)

        out2=self.encode1(self.dense_block1(out1))

        out3=self.encode2(self.dense_block2(out2))

        out4=self.encode3(self.dense_block3(out3))

        out5=self.encode4(self.dense_block4(out4))

       #############################################################################################
        out11=self.decode1(self.dense_block5(out5))
        out11=torch.cat([out4, out11], 1)
        out11=self.dense_block11(out11)
        out22=self.decode2(out11)
        out22=torch.cat([out3, out22], 1)
        out33=self.decode3(self.dense_block22(out22))
        out33=torch.cat([out33, out2], 1)
        out44=self.decode4(self.dense_block33(out33))
        out_V=self.dense_block44(out44)
        out_V=self.last_V(out_V)
        #Try different activations
        #self.activation=nn.Tanh()
        #self.activation=nn.Sigmoid()
        #out=self.scaled_shifted_tanh(out)
        self.activation=nn.Identity()
        out_V=self.activation(out_V)
        ############################################################






        return out_V



In [ ]:
# added layer
    from torchsummary import summary

    parser=argparse.ArgumentParser("Convolutional Encoder-Decoder networks")
    parser.add_argument('--in_channel', type=int, default=2,help='number of input channels')
    parser.add_argument('--out_channel_V',type=int, default=2, help='numebr of output channels')
    parser.add_argument('--out_channel_P',type=int, default=1, help='numebr of output channels')
    parser.add_argument('--growth_rate',type=int, default=8, help='equivalent with K=16')
    parser.add_argument('--init_features', type=int, default=4, help='number of channels after the first convolutional layer')
    parser.add_argument('--bn_size',type=int, default=8, help='battleneck size')
    parser.add_argument('--bottleneck',type=str, default=False, help='battleneck')
    parser.add_argument('--drop_rate', type=int, default=0, help='dropout probability')
    parser.add_argument('--kernel', type=int, default=[7,7,4,6],help='kernel size[k_conv,k,k_last,k_first]')
    parser.add_argument('--stride',type=int, default=[1,2,2,2], help='stride[s_conv,s,s_last,s_first]')
    parser.add_argument('--output_padding', type=int, default=[1,0,1,0],help='padding[encoder,med_layer,decoder,last]')
    parser.add_argument('--padding', type=int, default=[3,3,1,2],help='padding[p_conv,p,p_last.p_first]')
    args=parser.parse_args()


    device = 'cuda'
    ConvNet=model(args.in_channel,args.kernel,args.stride,args.padding,args.output_padding,args.out_channel_V,args.out_channel_P, args.init_features,args.growth_rate,args.bn_size,bottleneck=False,drop_out=0)
    ConvNet=ConvNet.to(device)
    input_size=400
    summary(ConvNet, (2, input_size, input_size))



In [ ]:
import torch.nn.init as init
#model initialization
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

## Sobel Filter

In [ ]:
class SobelFilter(object):

    def __init__(self, imsize, correct=True, device='cuda'):
        # conv2d is cross-correlation, need to transpose the kernel here
        self.HSOBEL_WEIGHTS_3x3 = torch.FloatTensor(
            np.array([[-1, -2, -1],
                     [ 0, 0, 0],
                     [1, 2, 1]])/(8*400) ).unsqueeze(0).unsqueeze(0).to(device)

        self.VSOBEL_WEIGHTS_3x3 = self.HSOBEL_WEIGHTS_3x3.transpose(-1, -2)

        self.VSOBEL_WEIGHTS_5x5 = torch.FloatTensor(
                    np.array([[-5, -4, 0, 4, 5],
                                [-8, -10, 0, 10, 8],
                                [-10, -20, 0, 20, 10],
                                [-8, -10, 0, 10, 8],
                                [-5, -4, 0, 4, 5]]) / (240*400)).unsqueeze(0).unsqueeze(0).to(device)
        self.HSOBEL_WEIGHTS_5x5 = self.VSOBEL_WEIGHTS_5x5.transpose(-1, -2)

        modifier = np.eye(imsize)
        modifier[0:2, 0] = np.array([4, -1])
        modifier[-2:, -1] = np.array([-1, 4])
        self.modifier = torch.FloatTensor(modifier).to(device)
        self.correct = correct


    def grad_h(self, image, filter_size=3):
        """Get image gradient along horizontal direction, or x axis.
        Option to do replicate padding for image before convolution.
        Args:
            image (Tensor): (1, 1, H, W)
            replicate_pad (None, int, 4-tuple): if 4-tuple, (padLeft, padRight, padTop,
                padBottom)
        """
        image_width = image.shape[-1]

        if filter_size == 3:
            replicate_pad = 1
            kernel = self.VSOBEL_WEIGHTS_3x3
        elif filter_size == 5:
            replicate_pad = 2
            kernel = self.VSOBEL_WEIGHTS_5x5

        image = F.pad(image, _quadruple(replicate_pad), mode='replicate')
        grad = F.conv2d(image, kernel, stride=1, padding=0, bias=None) * image_width
        # modify the boundary based on forward & backward finite difference (three points)
        # forward [-3, 4, -1], backward [3, -4, 1]
        if self.correct:
            return torch.matmul(grad, self.modifier)
        else:
            return grad

    def grad_v(self, image, filter_size=3):
        image_height = image.shape[-2]
        if filter_size == 3:
            replicate_pad = 1
            kernel = self.HSOBEL_WEIGHTS_3x3
        elif filter_size == 5:
            replicate_pad = 2
            kernel = self.HSOBEL_WEIGHTS_5x5
        image = F.pad(image, _quadruple(replicate_pad), mode='replicate')
        grad = F.conv2d(image, kernel, stride=1, padding=0,
            bias=None) * image_height
        # modify the boundary based on forward & backward finite difference
        if self.correct:
            return torch.matmul(self.modifier.t(), grad)
        else:
            return grad

In [ ]:
# continuity equation
def continuity(ux,vy):


    loss=ux+vy
    return loss

In [ ]:
# calculate gradients
def gradients(tensor,sobel):
    ux=sobel.grad_h(tensor[:,0:1,:,:], filter_size=5)
    uy=sobel.grad_v(tensor[:,0:1,:,:], filter_size=5)


    vx=sobel.grad_h(tensor[:,1:2,:,:], filter_size=5)
    vy=sobel.grad_v(tensor[:,1:2,:,:], filter_size=5)
    return ux,uy,vx,vy


In [ ]:
# satisfying boundray conditions
def BCs(ux,uy,vx,vy,outputs,targets):
    inlet=F.mse_loss(outputs[:,0:1,:,0],targets[:,0:1,:,0] , reduction='sum')+F.mse_loss(outputs[:,1:2,:,0],targets[:,1:2,:,0] , reduction='sum')
    outlet=F.mse_loss(ux[:,:,:,-1], torch.zeros_like(ux[:,:,:,-1]), reduction='sum')+F.mse_loss(vx[:,:,:,-1], torch.zeros_like(vx[:,:,:,-1]), reduction='sum')
    top=F.mse_loss(uy[:,:,:,-1], torch.zeros_like(uy[:,:,:,-1]), reduction='sum')+F.mse_loss(vy[:,:,:,-1], torch.zeros_like(vy[:,:,:,-1]), reduction='sum')
    bottom=F.mse_loss(uy[:,:,:,-1], torch.zeros_like(uy[:,:,:,-1]), reduction='sum')+F.mse_loss(vy[:,:,:,-1], torch.zeros_like(vy[:,:,:,-1]), reduction='sum')
    BC_loss=inlet+outlet+top+bottom
    return BC_loss

In [ ]:
# Navier Stokes equations
def Navier_Stokes(tensor,ux,uy,vx,vy,P,k,omega,force,sobel):
    nu=1.5*10**-5
    mu=1.837*10**-5
    rho=1.225
    nut=k/omega



    uxx=sobel.grad_h(ux, filter_size=5)
    uyy=sobel.grad_v(uy, filter_size=5)
    vyy=sobel.grad_v(vy, filter_size=5)
    vxx=sobel.grad_h(vx, filter_size=5)
    px=sobel.grad_h(P, filter_size=5)
    py=sobel.grad_v(P, filter_size=5)

    termA=tensor[:,0:1,:,:]*ux+tensor[:,1:2,:,:]*uy
    termB=(-1/rho)*px
    termC=nu*(uxx+uyy)
    termD=nut*(2*ux)-(2/3)*k

    termE=nut*(uy+vx)
    termDx=sobel.grad_h(termD)
    termEy=sobel.grad_v(termE)

    NSx=termB+termC+termDx+termEy-termA-force


    termAA=tensor[:,0:1,:,:]*vx+tensor[:,1:2,:,:]*vy
    termBB=(-1/rho)*py
    termCC=nu*(vxx+vyy)
    termDD=nut*(vx+uy)
    termEE=nut*(2*vy)-(2/3)*k
    termDDx=sobel.grad_h(termDD)
    termEEy=sobel.grad_v(termEE)
    NSy=termBB+termCC+termDDx+termEEy-termAA
#     loss_Nsx=(NSx**2).sum()
#     loss_Nsy=(NSy**2).sum()
    loss_Nsx=NSx
    loss_Nsy=NSy
    return loss_Nsx,loss_Nsy

## Training a physics-informed convolutional enocoder decoder

In [ ]:
# Train Function

def train(args,model,i,w_mse,w_mass,w_NSx,w_NSy,STEP):

    device='cuda'

    train_set=torch.utils.data.DataLoader(train_data, args.batch_size, pin_memory=True, num_workers=0)
    valid_set=torch.utils.data.DataLoader(valid_data, args.test_batch_size, pin_memory=True, num_workers=0)


    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate,
                       weight_decay=args.weight_decay)


#     scheduler = CosineAnnealingLR(optimizer, T_max=args.epochs, eta_min=0,last_epoch=-1)
#     scheduler=ReduceLROnPlateau(optimizer,mode='min',threshold_mode='rel', cooldown=0, min_lr=0.00001, eps=1e-8,patience=15,factor=0.5)

    scheduler = StepLR(optimizer, step_size=STEP, gamma=0.5)

    n_out_pixels_train = args.ntrain * train_data[0][1].numel()
    n_out_pixels_valid= args.nvalid * valid_data[0][1].numel()


    train_losses=[]
    train_losses_MSE=[]
    train_losses_Mass=[]
    valid_losses=[]
    valid_losses_log=[]
    valid_losses_mass=[]
    RMSE_valid=[]
    RMSE_train=[]
    learning_rates=[]


    Epochs=[]
    best_loss=1e9

    folder_path = r'./Minmax_input_No_output_gradual_BCs_NS_SSE/{}_{}_{}/{}'.format(w_mass,w_NSx,w_NSy,i)

# Create the folder
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)


    for epoch in range(args.epochs):
        # Train the Model

        model.train()  # Change model to 'train' mod

        train_loss=0
        train_loss_mse=0
        train_loss_mass=0
        train_loss_mse_log=0
        train_loss_BCs=0
        train_loss_Nsx=0
        train_loss_Nsy=0

        for step,(inputs,targets,P,K,Omega,force_tensor) in enumerate(train_set):

            inputs  = inputs.to(device, dtype=torch.float)
            targets= targets.to(device, dtype=torch.float)
            P= P.to(device, dtype=torch.float)
            K= K.to(device, dtype=torch.float)
            Omega= Omega.to(device, dtype=torch.float)
            force_tensor= force_tensor.to(device, dtype=torch.float)

            model.zero_grad()
            outputs=model(inputs)
            outputs= outputs.to(device, dtype=torch.float)

##################################################################################################
            Ux,Uy,Vx,Vy=gradients(outputs,SobelFilter(targets.shape[-1], correct=True))
            Ux_t,Uy_t,Vx_t,Vy_t=gradients(targets,SobelFilter(targets.shape[-1], correct=True))
            Boundray=BCs(Ux,Uy,Vx,Vy,outputs,targets)

            mass_o=continuity(Ux,Vy)
            mass_t=continuity(Ux_t,Vy_t)
            mass=F.mse_loss(mass_o,mass_t , reduction='sum')


            loss_nx_o,loss_ny_o=Navier_Stokes(outputs,Ux,Uy,Vx,Vy,P,K,Omega,force_tensor,SobelFilter(targets.shape[-1], correct=True))
            loss_nx_t,loss_ny_t=Navier_Stokes(outputs,Ux_t,Uy_t,Vx_t,Vy_t,P,K,Omega,force_tensor,SobelFilter(targets.shape[-1], correct=True))
            loss_nx=F.mse_loss(loss_nx_o,loss_nx_t , reduction='sum')
            loss_ny=F.mse_loss(loss_ny_o,loss_ny_t , reduction='sum')

            loss_Mass=torch.log(1+mass)
            loss_BCs=torch.log(1+Boundray)
            loss_NX=torch.log(1+loss_nx)
            loss_NY=torch.log(1+loss_ny)
            train_loss_mass+=loss_Mass.item()
            train_loss_BCs+=loss_BCs.item()
            train_loss_Nsx+=loss_NX.item()
            train_loss_Nsy+=loss_NY.item()

#####################################################################################################




            loss_MSE=F.mse_loss(outputs,targets , reduction='sum')


            loss_MSE_log=torch.log(1+loss_MSE)

#             loss_MSE_log=loss_MSE
            w0=(w_mass/args.epochs)*epoch
#             w0=w_mass
            w00=(w_NSx/args.epochs)*epoch
            w000=(w_NSy/args.epochs)*epoch



            loss=w_mse*loss_MSE_log+w0*loss_Mass+w0*loss_BCs+w00*loss_NX+w000*loss_NY




            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)

            optimizer.step()




            train_loss+=loss.item()
#
            train_loss_mse_log+=loss_MSE_log.item()
            train_loss_mse+=loss_MSE.item()


        train_losses.append(train_loss)
        train_losses_MSE.append(train_loss_mse)
        train_losses_Mass.append(train_loss_mass)


        rmse = np.sqrt(train_loss_mse / n_out_pixels_train)

        RMSE_train.append(rmse)


        lr_rate=optimizer.param_groups[0]['lr']

        learning_rates.append(lr_rate)
#         writer.add_scalar("loss/lr_rate", lr_rate, epoch)

        Epochs.append(epoch+1)
        model.eval()
        validation_loss,validation_loss_log=evaluate(model,epoch,valid_set)
#         writer.add_scalar("loss/validation_loss", validation_loss_log, epoch)
        rmse_val = np.sqrt(validation_loss / n_out_pixels_valid)



        valid_losses.append(validation_loss)
        valid_losses_log.append(validation_loss_log)
        RMSE_valid.append(rmse_val)
        scheduler.step()




        if epoch>=200:
            if validation_loss < best_loss:
                best_loss = validation_loss



                torch.save(model.state_dict(), folder_path+'/model_data_{}_lr_{}_epoch_{}_batch_{}_step_{}_{}.pt'.format(ntrain,args.learning_rate,args.epochs,args.batch_size,STEP,i))
#







        if epoch % 10 == 0:
            print(("Epoch {}: , Train RMSE : {},Validation RMSE : {}").format(
                      epoch ,rmse,rmse_val))

            print(("Epoch {}: ,Train Loss:{}, Train Mass: {}, Train MSE: {}, Valid MSE:{}").format(
                       epoch ,train_loss,train_loss_mass,train_loss_mse_log,validation_loss))


        if epoch < (args.epochs-50):
            if args.plot:
                visual(targets, outputs, args.gpu, 1)








    if args.plot_LC:

        plt.title("Training Curve")
        plt.plot(Epochs, train_losses_MSE, label="Train")

        plt.plot(Epochs, valid_losses, label="Valid")


        plt.legend(loc='best')
        plt.xlabel("Iterations")
        plt.ylabel("Loss")
        plt.show()
        plt.title("RMSE")
        plt.plot(Epochs, RMSE_train, label="Train")

        plt.plot(Epochs, RMSE_valid, label="Valid")




        plt.legend(loc='best')
        plt.xlabel("Iterations")
        plt.ylabel("RMSE")
        plt.show()

    return model


In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
args = AttrDict()

## Train and Save Model

In [ ]:

args_dict = {"gpu": True,
                                                "valid": False,
                                                "plot_LC": True,
                                                "checkpoint": "",
                                                "plot": False,
                                                "visualize": False,
                                                "downsize_input": False,
                                                "batch_size":64,
                                                "test_batch_size":20,
                                                "learning_rate":0.008,
                                                "weight_decay":0.005,
                                                "epochs":300,
                                                "ntrain":ntrain,
                                                "nvalid":nvalid,
}
args.update(args_dict)
w_mse=1
w_mass=0.02
w_NSx=0.01
w_NSy=0.01
i=0
model=train_with_cross_validation(args, i, w_mse, w_mass, w_NSx, w_NSy, num_folds=10)